In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, transpile, assemble, circuit
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram
import numpy as np

In [2]:
def gcd(a,b):
    if a < b:
        return gcd(b,a)
    if b == 0:
        return a
    return gcd(a%b, b)

In [3]:
def mod_by_rep_square(num, repetitions, modulus):
    for i in range(repetitions):
        num = np.mod(num**2, modulus)
    return num

In [4]:
def exponent_to_factors(guess, exponent, num):
    if exponent % 2 == 1:
        return (1, num)
    else:
        return (gcd(guess ** exponent + 1, num), gcd(guess**exponent - 1, num))

In [28]:
def ADD_circuit(bits, add):
    """ Creates a Quantum Circuit that increments a register of qubits in the Fourier Space by the parameter add """
    c_add = QuantumCircuit(bits)
    bin_add = bin(add)[2:]
    while len(bin_add) < bits:
        bin_add = '0' + bin_add
    for cbit in range(len(bin_add)):
        if bin_add[cbit] == '1':
            for qbit in range(bits):
                c_add.p( (np.pi/(2**(cbit - qbit))), bits - qbit - 1)
    return c_add 
def ADD(bits, add):   
    c_add = ADD_circuit(bits, add).to_instruction()
    c_add.name = "add {}".format(add)
    return c_add
def CADD(bits, add):
    c_add = ADD_circuit(bits, add)
    c_add.name = "add {}".format(add)
    return c_add.control(num_ctrl_qubits=1).to_instruction()

In [29]:
def test_ADD(n1, n2, bits):
    """ Use QFT, classical_add, and Inverse QFT to add two numbers """
    if n1 > (2**(bits - 2)) or n2 > (2**(bits - 2)):
        raise OverflowError
    bn1 = bin(n1)[2:]
    while len(bn1) < bits:
        bn1 = '0' + bn1
    qr = QuantumRegister(bits, "q")
    qc = QuantumCircuit(qr)
    for bit in range(len(bn1)):
        if bn1[bit] == '1':
            qc.x(qr[bits - bit - 1])
    where = []
    for i in range(len(qr)):
        where.append(qr[i])
    qc.append(circuit.library.QFT(num_qubits = bits, do_swaps = False).to_instruction(), where)
    qc.append(ADD(bits, n2), where)
    qc.append(circuit.library.QFT(num_qubits = bits, do_swaps = False).inverse().to_instruction(), where)
    qc.measure_all()
    return qc

In [30]:
q = test_ADD(1, 4, 4)
q.draw()

┌───┐┌──────┐┌────────┐┌───────┐ ░ ┌─┐         
   q_0: ┤ X ├┤0     ├┤0       ├┤0      ├─░─┤M├─────────
        └───┘│      ││        ││       │ ░ └╥┘┌─┐      
   q_1: ─────┤1     ├┤1       ├┤1      ├─░──╫─┤M├──────
             │  qft ││  add 4 ││  iqft │ ░  ║ └╥┘┌─┐   
   q_2: ─────┤2     ├┤2       ├┤2      ├─░──╫──╫─┤M├───
             │      ││        ││       │ ░  ║  ║ └╥┘┌─┐
   q_3: ─────┤3     ├┤3       ├┤3      ├─░──╫──╫──╫─┤M├
             └──────┘└────────┘└───────┘ ░  ║  ║  ║ └╥┘
meas: 4/════════════════════════════════════╩══╩══╩══╩═
                                            0  1  2  3

In [31]:
backend = QasmSimulator()
q_comp = transpile(q, backend)
job = backend.run(q_comp, shots = 10)
result = job.result()
result.get_counts(q_comp)

{'0101': 10}

In [32]:
def MODADD(b, m, bits):
    """ Use the Quantum_Add Gate to add to a number in the modular base """
    qr = QuantumRegister(bits, "q")
    ar = QuantumRegister(1, "a")
    qc = QuantumCircuit(qr, ar)
    where = []
    for i in range(bits):
        where.append(qr[i])
    qc.append(ADD(bits, b), where)
    qc.append(ADD(bits, m).inverse(), where)
    qc.append(circuit.library.QFT(num_qubits=bits, do_swaps=False).inverse().to_instruction(), where)
    qc.cx(qr[bits-1],ar[0])
    qc.append(circuit.library.QFT(num_qubits=bits, do_swaps=False).to_instruction(), where)
    where2 = [ar[0]] + where
    qc.append(CADD(bits, m), where2)
    qc.append(ADD(bits, b).inverse(), where)
    qc.append(circuit.library.QFT(num_qubits=bits, do_swaps=False).inverse().to_instruction(), where)
    qc.x(qr[bits-1])
    qc.cx(qr[bits-1],ar[0])
    qc.x(qr[bits-1])
    qc.append(circuit.library.QFT(num_qubits=bits, do_swaps=False).to_instruction(), where)
    qc.append(ADD(bits, b), where)
    return qc

In [33]:
q = MODADD(4, 10, 5)
q.draw()

┌────────┐┌────────────┐┌───────┐     ┌──────┐┌───────────┐┌───────────┐»
q_0: ┤0       ├┤0           ├┤0      ├─────┤0     ├┤1          ├┤0          ├»
     │        ││            ││       │     │      ││           ││           │»
q_1: ┤1       ├┤1           ├┤1      ├─────┤1     ├┤2          ├┤1          ├»
     │        ││            ││       │     │      ││           ││           │»
q_2: ┤2 add 4 ├┤2 add 10_dg ├┤2 iqft ├─────┤2 qft ├┤3          ├┤2 add 4_dg ├»
     │        ││            ││       │     │      ││  c_add 10 ││           │»
q_3: ┤3       ├┤3           ├┤3      ├─────┤3     ├┤4          ├┤3          ├»
     │        ││            ││       │     │      ││           ││           │»
q_4: ┤4       ├┤4           ├┤4      ├──■──┤4     ├┤5          ├┤4          ├»
     └────────┘└────────────┘└───────┘┌─┴─┐└──────┘│           │└───────────┘»
a_0: ─────────────────────────────────┤ X ├────────┤0          ├─────────────»
                                      └───┘        └───────────┘             »
«     ┌───────┐               ┌──────┐┌────────┐
«q_0: ┤0      ├───────────────┤0     ├┤0       ├
«     │       │               │      ││        │
«q_1: ┤1      ├───────────────┤1     ├┤1       ├
«     │       │               │      ││        │
«q_2: ┤2 iqft ├───────────────┤2 qft ├┤2 add 4 ├
«     │       │               │      ││        │
«q_3: ┤3      ├───────────────┤3     ├┤3       ├
«     │       │┌───┐     ┌───┐│      ││        │
«q_4: ┤4      ├┤ X ├──■──┤ X ├┤4     ├┤4       ├
«     └───────┘└───┘┌─┴─┐└───┘└──────┘└────────┘
«a_0: ──────────────┤ X ├───────────────────────
«                   └───┘

In [34]:
def test_MODADD(a, b, m, bits):
    """ Function to test the efficacy of the modular_add_gate"""
    bina = bin(a)[2:]
    while(len(bina) < bits):
        bina = '0' + bina
    qr = QuantumRegister(bits, "q")
    ar = QuantumRegister(1, "a")
    qc = QuantumCircuit(qr, ar)
    for bit in range(len(bina)):
        if bina[bit] == '1':
            qc.x(qr[bits - bit - 1])
    where = []
    for i in range(len(qr)):
        where.append(qr[i])
    qc.append(circuit.library.QFT(num_qubits=bits, do_swaps=False).to_instruction(), where)
    where2 = where + [ar[0]]
    qc.append(MODADD(b, m, bits).to_instruction(), where2)
    qc.append(circuit.library.QFT(num_qubits=bits, do_swaps=False).inverse().to_instruction(), where)
    qc.measure_all()
    return qc
q = test_MODADD(1,4,6,6)
q.draw()
    

┌───┐┌──────┐┌──────────────┐┌───────┐ ░ ┌─┐                  
   q_0: ┤ X ├┤0     ├┤0             ├┤0      ├─░─┤M├──────────────────
        └───┘│      ││              ││       │ ░ └╥┘┌─┐               
   q_1: ─────┤1     ├┤1             ├┤1      ├─░──╫─┤M├───────────────
             │      ││              ││       │ ░  ║ └╥┘┌─┐            
   q_2: ─────┤2     ├┤2             ├┤2      ├─░──╫──╫─┤M├────────────
             │  qft ││              ││  iqft │ ░  ║  ║ └╥┘┌─┐         
   q_3: ─────┤3     ├┤3 circuit-238 ├┤3      ├─░──╫──╫──╫─┤M├─────────
             │      ││              ││       │ ░  ║  ║  ║ └╥┘┌─┐      
   q_4: ─────┤4     ├┤4             ├┤4      ├─░──╫──╫──╫──╫─┤M├──────
             │      ││              ││       │ ░  ║  ║  ║  ║ └╥┘┌─┐   
   q_5: ─────┤5     ├┤5             ├┤5      ├─░──╫──╫──╫──╫──╫─┤M├───
             └──────┘│              │└───────┘ ░  ║  ║  ║  ║  ║ └╥┘┌─┐
   a_0: ─────────────┤6             ├──────────░──╫──╫──╫──╫──╫──╫─┤M├
                     └──────────────┘          ░  ║  ║  ║  ║  ║  ║ └╥┘
meas: 7/══════════════════════════════════════════╩══╩══╩══╩══╩══╩══╩═
                                                  0  1  2  3  4  5  6

In [35]:
q_comp = transpile(q, backend)
job = backend.run(q_comp, shots = 10)
result = job.result()
result.get_counts(q_comp)

{'0000101': 10}

In [36]:
def DCMODADD(b, m, bits):
    """ Doubly Controlled Modular Addition Gate (in the Fourier Space)"""
    qc = MODADD(b, m, bits)
    qc = qc.decompose()
    qc.name = "add {} mod {}".format(b, m)
    return qc.control(num_ctrl_qubits = 2)

In [37]:
qc = DCMODADD(2, 4, 7)
qc.draw()

q153_0: ───────■────────
               │        
q153_1: ───────■────────
        ┌──────┴───────┐
   q_0: ┤0             ├
        │              │
   q_1: ┤1             ├
        │              │
   q_2: ┤2             ├
        │              │
   q_3: ┤3             ├
        │  add 2 mod 4 │
   q_4: ┤4             ├
        │              │
   q_5: ┤5             ├
        │              │
   q_6: ┤6             ├
        │              │
   a_0: ┤7             ├
        └──────────────┘

In [38]:
def AFFMULT(a, m, bits):
    """ Given an x in the computational basis, return ax + the value in the b register in the Fourier Space"""
    cr = QuantumRegister(1, 'c')
    xr = QuantumRegister(bits, 'x')
    br = QuantumRegister(bits, 'b')
    ar = QuantumRegister(1, 'a')
    qc = QuantumCircuit(cr, xr, br, ar)
    br_list = []
    for i in range(bits):
        br_list.append(br[i])
    for i in range(bits):
        where = br_list
        where = [cr[0]] + [xr[i]] + br_list + [ar[0]]
        qc.append(DCMODADD((2 ** i) * a, m, bits), where)
    q = qc.to_instruction()
    q.name = "multiply {} mod {}".format(a, m)
    return q

In [39]:
def test_AFFMULT(a, x, b, m, bits):
    """ Function to test the efficacy of the CMULT Gate"""
    cr = QuantumRegister(1, 'c')
    xr = QuantumRegister(bits, 'x')
    br = QuantumRegister(bits, 'b')
    ar = QuantumRegister(1, 'a')
    qc = QuantumCircuit(cr, xr, br, ar)
    binx = bin(x)[2:]
    while(len(binx) < bits):
        binx = '0' + binx
    for bit in range(len(binx)):
        if binx[bit] == '1':
            qc.x(xr[bits - bit - 1])
    binb = bin(b)[2:]
    while(len(binb) < bits):
        binb = '0' + binb
    for bit in range(len(binb)):
        if binb[bit] == '1':
            qc.x(br[bits - bit - 1])
    br_list = []
    for i in range(bits):
        br_list.append(br[i])
    qc.x(cr[0])
    qc.append(circuit.library.QFT(num_qubits=bits, do_swaps=False).to_instruction(), br_list)
    qc.append(AFFMULT(a, m, bits), range(2 * bits + 2))
    qc.append(circuit.library.QFT(num_qubits=bits, do_swaps=False).inverse().to_instruction(), br_list)
    qc.measure_all()
    return qc
qc = test_AFFMULT(1, 4, 1, 4, 4)
qc.draw()

┌───┐        ┌───────────────────┐          ░ ┌─┐                     »
    c_0: ┤ X ├────────┤0                  ├──────────░─┤M├─────────────────────»
         └───┘        │                   │          ░ └╥┘┌─┐                  »
    x_0: ─────────────┤1                  ├──────────░──╫─┤M├──────────────────»
                      │                   │          ░  ║ └╥┘┌─┐               »
    x_1: ─────────────┤2                  ├──────────░──╫──╫─┤M├───────────────»
         ┌───┐        │                   │          ░  ║  ║ └╥┘┌─┐            »
    x_2: ┤ X ├────────┤3                  ├──────────░──╫──╫──╫─┤M├────────────»
         └───┘        │                   │          ░  ║  ║  ║ └╥┘┌─┐         »
    x_3: ─────────────┤4                  ├──────────░──╫──╫──╫──╫─┤M├─────────»
         ┌───┐┌──────┐│  multiply 1 mod 4 │┌───────┐ ░  ║  ║  ║  ║ └╥┘┌─┐      »
    b_0: ┤ X ├┤0     ├┤5                  ├┤0      ├─░──╫──╫──╫──╫──╫─┤M├──────»
         └───┘│      ││                   ││       │ ░  ║  ║  ║  ║  ║ └╥┘┌─┐   »
    b_1: ─────┤1     ├┤6                  ├┤1      ├─░──╫──╫──╫──╫──╫──╫─┤M├───»
              │  qft ││                   ││  iqft │ ░  ║  ║  ║  ║  ║  ║ └╥┘┌─┐»
    b_2: ─────┤2     ├┤7                  ├┤2      ├─░──╫──╫──╫──╫──╫──╫──╫─┤M├»
              │      ││                   ││       │ ░  ║  ║  ║  ║  ║  ║  ║ └╥┘»
    b_3: ─────┤3     ├┤8                  ├┤3      ├─░──╫──╫──╫──╫──╫──╫──╫──╫─»
              └──────┘│                   │└───────┘ ░  ║  ║  ║  ║  ║  ║  ║  ║ »
    a_0: ─────────────┤9                  ├──────────░──╫──╫──╫──╫──╫──╫──╫──╫─»
                      └───────────────────┘          ░  ║  ║  ║  ║  ║  ║  ║  ║ »
meas: 10/═══════════════════════════════════════════════╩══╩══╩══╩══╩══╩══╩══╩═»
                                                        0  1  2  3  4  5  6  7 »
«               
«    c_0: ──────
«               
«    x_0: ──────
«               
«    x_1: ──────
«               
«    x_2: ──────
«               
«    x_3: ──────
«               
«    b_0: ──────
«               
«    b_1: ──────
«               
«    b_2: ──────
«         ┌─┐   
«    b_3: ┤M├───
«         └╥┘┌─┐
«    a_0: ─╫─┤M├
«          ║ └╥┘
«meas: 10/═╩══╩═
«          8  9

In [40]:
q_comp = transpile(qc, backend)
job = backend.run(q_comp, shots = 10)
result = job.result()
out = [*result.get_counts(q_comp)]
for i in out:
    print(i[1:5])
    #get only the results of registers b

0001


In [41]:
def CSWAP():
    ar = QuantumRegister(1)
    br = QuantumRegister(1)
    cr = QuantumRegister(1)
    qc = QuantumCircuit(cr, ar, br)
    qc.cx(br[0], ar[0])
    qc.toffoli(cr[0], ar[0], br[0])
    qc.cx(br[0], ar[0])
    return qc
def CSWAPREG(bits):
    ar = QuantumRegister(bits)
    br = QuantumRegister(bits)
    cr = QuantumRegister(1)
    qc = QuantumCircuit(cr, ar, br)
    for i in range(bits):
        qc.append(CSWAP(), [cr[0], ar[i], br[i]])
    q = qc.to_instruction()
    q.name = "c_SWAPREG"
    return q

In [42]:
def test_CSWAPREG(b, a, bits):
    ar = QuantumRegister(bits, 'a')
    br = QuantumRegister(bits, 'b')
    cr = QuantumRegister(1, 'c')
    qc = QuantumCircuit(cr, ar, br)
    qc.x(cr[0])
    bina = bin(a)[2:]
    binb = bin(b)[2:]
    while len(bina) < bits:
        bina = '0' + bina
    while len(binb) < bits:
        binb = '0' + binb
    for bit in range(len(bina)):
        if bina[bit] == '1':
            qc.x(ar[bits - bit - 1])
    for bit in range(len(binb)):
        if binb[bit] == '1':
            qc.x(br[bits - bit - 1])
    qc.append(CSWAPREG(bits), range(2*bits + 1))
    return qc
a = test_CSWAPREG(2, 3, 4)
a.measure_all()
a.draw()

┌───┐┌────────────┐ ░ ┌─┐                        
   c_0: ┤ X ├┤0           ├─░─┤M├────────────────────────
        ├───┤│            │ ░ └╥┘┌─┐                     
   a_0: ┤ X ├┤1           ├─░──╫─┤M├─────────────────────
        ├───┤│            │ ░  ║ └╥┘┌─┐                  
   a_1: ┤ X ├┤2           ├─░──╫──╫─┤M├──────────────────
        └───┘│            │ ░  ║  ║ └╥┘┌─┐               
   a_2: ─────┤3           ├─░──╫──╫──╫─┤M├───────────────
             │            │ ░  ║  ║  ║ └╥┘┌─┐            
   a_3: ─────┤4 c_SWAPREG ├─░──╫──╫──╫──╫─┤M├────────────
             │            │ ░  ║  ║  ║  ║ └╥┘┌─┐         
   b_0: ─────┤5           ├─░──╫──╫──╫──╫──╫─┤M├─────────
        ┌───┐│            │ ░  ║  ║  ║  ║  ║ └╥┘┌─┐      
   b_1: ┤ X ├┤6           ├─░──╫──╫──╫──╫──╫──╫─┤M├──────
        └───┘│            │ ░  ║  ║  ║  ║  ║  ║ └╥┘┌─┐   
   b_2: ─────┤7           ├─░──╫──╫──╫──╫──╫──╫──╫─┤M├───
             │            │ ░  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐
   b_3: ─────┤8           ├─░──╫──╫──╫──╫──╫──╫──╫──╫─┤M├
             └────────────┘ ░  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘
meas: 9/═══════════════════════╩══╩══╩══╩══╩══╩══╩══╩══╩═
                               0  1  2  3  4  5  6  7  8

In [13]:
q_comp = transpile(a, backend)
job = backend.run(q_comp, shots = 10)
result = job.result()
result.get_counts(q_comp)

{'001100101': 10}